In [7]:
import os

import sagemaker
from sagemaker.pytorch import PyTorch

## Upload Data

In [38]:
session = sagemaker.Session()

bucket = session.default_bucket()
s3_prefix = 'capstone/v1'

role = sagemaker.get_execution_role()

In [ ]:
data_dir = 'processed'
input_data = session.upload_data(path=data_dir, bucket=bucket, key_prefix=s3_prefix)

## Train the Model

In [22]:
output_path = f's3://{bucket}/{s3_prefix}/output'

estimator = PyTorch(
    source_dir='src',
    entry_point='train.py',
    role=role,
    framework_version='1.1.0',
    py_version='py3',
    train_instance_count=1,
    train_instance_type='ml.p2.xlarge',
    output_path=output_path,
    hyperparameters={
        'seed': 1,
        'batch-size': 512,
        'epochs': 100,
        'embedding-dim': 32,
        'num-lstm-layers': 1,
        'hidden-dims': '100',
        # vocab size from previous step + 2 for
        # out of vocab and empty
        'vocab-size': 10002
    }
)

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [26]:
estimator.fit({
    'training': os.path.join(input_data, 'test'),
    'eval': os.path.join(input_data, 'val')
})

2021-03-21 20:28:13 Starting - Starting the training job...
2021-03-21 20:28:37 Starting - Launching requested ML instancesProfilerReport-1616358492: InProgress
......
2021-03-21 20:29:37 Starting - Preparing the instances for training.........
2021-03-21 20:31:04 Downloading - Downloading input data......
2021-03-21 20:32:07 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-03-21 20:32:09,105 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2021-03-21 20:32:09,135 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-03-21 20:32:09,760 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-03-21 20:32:10,063 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2021-03-21 20:32:10,064 sagemaker-containers INFO    

## Predict

In [ ]:
# Deploy my estimator to a SageMaker Endpoint and get a Predictor
predictor = estimator.deploy(
    instance_type='ml.m4.xlarge',
    initial_instance_count=1
)

In [36]:
from importlib import reload  
import src.utils
reload(src.utils)
import joblib
with open('processed/vocab.joblib', 'rb') as f:
    vocab = joblib.load(f)

# `data` is a NumPy array or a Python list.
# `response` is a NumPy array.
sentence = 'You bad, you are the worst person alive!!!'
input_vec = src.utils.encode_single_input(sentence, vocab)
response = predictor.predict(input_vec)

In [ ]:
for 

In [ ]:
predictor.delete_endpoint()